In [211]:
import requests
from bs4 import BeautifulSoup
#import time
import csv
import json
import urllib
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

# Data Collection using APIs (version 2)

##### This version is still imperfect (timewise), we used an API to collect the game id of all games on steam, and then we proceeded by retrieving all the reviews for each game using a second API.
##### For the first part, we used selenium to get the game id (Code will be included), but we find a more efficient and easy way through a simple API.
##### As for the second part, Steam also provides an API to retrieve reviews about each game, so we used it as well. And then, we saved all the scraped information in two csv files; one containing overall review information about each game, and the other specific details of all reviews.
#### Another API made  by steam spy is used to retireve the genre,publisher, and price of a game

In [212]:
game_id_info = requests.get('http://api.steampowered.com/ISteamApps/GetAppList/v2/')
game_id_info = json.loads(game_id_info.text)
games_id = game_id_info['applist']['apps']

In [213]:
games_id

[{'appid': 216938, 'name': 'Pieterw test app76 ( 216938 )'},
 {'appid': 660010, 'name': 'test2'},
 {'appid': 660130, 'name': 'test3'},
 {'appid': 1118314, 'name': ''},
 {'appid': 1116300, 'name': 'Bounce'},
 {'appid': 1116310,
  'name': 'American Truck Simulator - Space Paint Jobs Pack'},
 {'appid': 1116330,
  'name': 'Nowhere Prophet Original Soundtrack + The Art and Design of Nowhere Prophet Book'},
 {'appid': 1116350, 'name': 'Exsys'},
 {'appid': 1116360, 'name': 'JERRY JOBHOPPER'},
 {'appid': 1115540, 'name': 'Volume Up'},
 {'appid': 1115570, 'name': 'Forest Mage'},
 {'appid': 1115580, 'name': 'Grand Brix Shooter'},
 {'appid': 1115600, 'name': 'Deep Diving Simulator - Adventure Pack'},
 {'appid': 1115610, 'name': 'The Lost and The Wicked'},
 {'appid': 1115620, 'name': 'Super Pig'},
 {'appid': 1115630, 'name': 'Magnetic Daydream'},
 {'appid': 1115640, 'name': '英雄群侠传II'},
 {'appid': 1115650, 'name': 'Blind Date 3D'},
 {'appid': 1115660, 'name': 'Apollo 11 VR HD: First Steps'},
 {'app

## Preparing the datasets

In [265]:
df_game_id = pd.DataFrame(columns=['game_id',
                                    'game_name',
                                   'publisher',
                                   'genre',
                                   'price',
                                   'initialprice',
                                   'overall_score',
                                     'score_desc',
                                     'total_positive',
                                     'total_negative',
                                     'total_reviews'])
df_game_id.to_csv('new_start_general.csv',mode='w',header=True)

In [372]:
df_game_id_reviews = pd.DataFrame(columns = ['game_id',
                                            'review_id',
                                           'num_games_owned',
                                           'num_reviews',
                                           'playtime_forever',
                                           'playtime_last_two_weeks',
                                           'playtime_at_review',
                                           'last_played',
                                           'review_text',
                                           'review_score',
                                           'steam_purchase',
                                           'comment_count',
                                            'voted_up',
                                            'votes_up',
                                            'votes_funny',
                                           'written_during_early_access',
                                            'review_type'])
df_game_id_reviews.to_csv('new_start_reviews.csv', mode='w', header=True)

## Defining a collection script

In [371]:
#With this script, we can collect positive and negative reviews separately. we added a list called done_games
#to save the games we scrapped in case of code interruption or problems
def collect_reviews(gameid,df_game_id_reviews,dataset,type_,done_games):
    cursor = '*'
    cursors_set = set()
    while True:
        url = "https://store.steampowered.com/appreviews/{}?json=1&filter=recent&review_type={}&num_per_page=100&cursor={}".format(gameid,type_,cursor)
        print(url)
        page = requests.get(url)
        page.encoding = 'utf-8-sig'
        if 'json' in page.headers.get('Content-Type') and page.text != '':
            review_info = json.loads(page.text)
            reviews = review_info['reviews']
            if len(reviews) > 0:
                for review in reviews:
                    reviewid= review['recommendationid'] if 'recommendationid' in review.keys() else None 
                    num_games_owned=review['author']['num_games_owned'] if 'num_games_owned' in review['author'].keys() else None 
                    num_reviews=review['author']['num_reviews'] if 'num_reviews' in review['author'].keys() else None 
                    playtime_forever=review['author']['playtime_forever'] if 'playtime_forever' in review['author'].keys() else None 
                    playtime_last_two_weeks=review['author']['playtime_last_two_weeks'] if 'playtime_last_two_weeks' in review['author'].keys() else None
                    playtime_at_review=review['author']['playtime_at_review'] if 'playtime_at_review' in review['author'].keys() else None
                    last_played=review['author']['last_played']  if 'last_played' in review['author'].keys() else None
                    review_score=review['weighted_vote_score'] if 'weighted_vote_score' in review.keys() else None
                    steam_purchase=review['steam_purchase'] if 'steam_purchase' in review.keys() else None
                    written_during_early_access=review['written_during_early_access'] if 'written_during_early_access' in review.keys() else None
                    comment_count= review['comment_count'] if 'comment_count' in review.keys() else None 
                    voted_up= review['voted_up'] if 'voted_up' in review.keys() else None 
                    votes_up= review['votes_up'] if 'votes_up' in review.keys() else None
                    votes_funny = review['votes_funny'] if 'votes_funny' in review.keys() else None
                    review_text = review['review'] if 'review' in review.keys() else None
                    developer_response = review['developer_response'] if 'developer_response' in review.keys() else None
                    data_review_entry = pd.Series(data = {'game_id':game_id,
                                                          'review_id':reviewid,
                                                          'num_games_owned':num_games_owned,
                                                          'num_reviews':num_reviews,
                                                          'playtime_forever':playtime_forever,
                                                          'playtime_last_two_weeks':playtime_last_two_weeks,
                                                          'playtime_at_review':playtime_at_review,
                                                          'last_played':last_played,
                                                          'review_text':review_text,
                                                          'review_score':review_score,
                                                          'steam_purchase':steam_purchase,
                                                          'comment_count':comment_count,
                                                          'voted_up':voted_up,
                                                          'votes_up':votes_up,
                                                          'votes_funny':votes_funny,
                                                          'written_during_early_access':written_during_early_access,
                                                         'review_type':type_})
                    #data_review_entry.to_csv('reviews2.csv', mode='a', header=False,)
                    df_game_id_reviews = df_game_id_reviews[0:0]
                    df_game_id_reviews = df_game_id_reviews.append(data_review_entry,ignore_index=True)
                    df_game_id_reviews.to_csv(dataset, mode='a', header=False,na_rep=np.nan)
            else:
                print("no {} reviews for this game".format(type_))
            
            cursor = urllib.parse.urlencode({'cursor' : review_info['cursor']}).split('=')[1]
            if cursor in cursors_set:
                break
            else:
                cursors_set.add(cursor)
            
        else:
            print('Response content is not in JSON format.')
            review_info = 'problem with api'
        done_games.add(gameid)

In [375]:
#loading the first dataset so that we can run the collection script with games id inside this dataset
df = pd.read_csv("new_start_general.csv")
df['game_id']

0       1116300
1       1116310
2       1116330
3       1116350
4       1116360
         ...   
6198     204840
6199     204860
6200     204880
6201     204900
6202     204901
Name: game_id, Length: 6203, dtype: int64

In [ ]:
#run these 3 cells if the data collection was interrupted.
uselesss = []
for game in useless_games:
    uselesss.append(game['appid'])

In [ ]:
last_games = []
done_games = np.array(df['game_id'])
for game in games_id:
    if game['appid'] not in done_games and game['appid'] not in uselesss:
        last_games.append(game)
len(last_games)

In [351]:
from operator import itemgetter
newlist = sorted(last_games, key=itemgetter('appid'), reverse=False)

In [ ]:
#for each game, we will retreive the needed information and save it directly to the csv file
for app in newlist:
    game_id = app['appid']
    game_name = app['name']
    print(game_id,game_name)
    cursor = '*'
    url_1 = "https://store.steampowered.com/appreviews/{}?json=1&cursor={}".format(game_id,cursor)
    print(url_1)
    url_2 = "https://steamspy.com/api.php?request=appdetails&appid={}".format(game_id)
    page_1 = requests.get(url_1)
    page_2 = requests.get(url_2)
    if 'json' in page_2.headers.get('Content-Type') and 'json' in page_1.headers.get('Content-Type') and page_2.text != '' and page_1.text != '':
        page_1.encoding = 'utf-8-sig'
        review_info =page_1.json()
        page_2.encoding = 'utf-8-sig'
        review_info_steamspy = json.loads(page_2.text)
        #scrapping basic information
        summary = review_info['query_summary']
        overall_score = summary['review_score'] if 'review_score' in summary.keys() else None 
        score_desc = summary['review_score_desc'] if 'review_score_desc' in summary.keys() else None 
        total_positive = summary['total_positive'] if 'total_positive' in summary.keys() else None 
        total_negative = summary['total_negative'] if 'total_negative' in summary.keys() else None 
        total_reviews = summary['total_reviews'] if 'total_reviews' in summary.keys() else None
        publisher = None
        genre = None
        price = None
        initialprice = None
        if overall_score != 0:
            if review_info_steamspy['name'] != None:
                publisher = review_info_steamspy['publisher'] if 'publisher' in review_info_steamspy.keys() else None
                genre = review_info_steamspy['genre'] if 'genre' in review_info_steamspy.keys() else None
                price = review_info_steamspy['price'] if 'price' in review_info_steamspy.keys() else None
                initialprice = review_info_steamspy['initialprice '] if 'initialprice ' in review_info_steamspy.keys() else None

            
            data_entry = pd.Series(data = {'game_id': game_id,
                         'game_name':game_name,
                         'publisher':publisher,
                         'genre': genre,
                         'price': price,
                         'initialprice': initialprice,
                         'overall_score': overall_score,
                         'score_desc': score_desc,
                         'total_positive': total_positive,
                         'total_negative': total_negative,
                         'total_reviews': total_reviews
                         })
            df_game_id =  df_game_id[0:0]
            df_game_id = df_game_id.append(data_entry,ignore_index=True)
            df_game_id.to_csv('new_start_general.csv',mode='a',header=False,na_rep=np.nan)
        else:
            useless_games.append(app)

    else:
        print('Response content is not in JSON format.')
        review_info = 'problem with api'

In [382]:
#Collecting reviews 
#done_games = set()
for game_id in anotherlist:
    print(game_id,len(done_games))
    collect_reviews(game_id,df_game_id_reviews,'new_start_reviews.csv','positive',done_games)
    collect_reviews(game_id,df_game_id_reviews,'new_start_reviews.csv','negative',done_games)

1002 4493
https://store.steampowered.com/appreviews/1002?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1002?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwoZaPiKwCcYGYBA%3D%3D
no positive reviews for this game
https://store.steampowered.com/appreviews/1002?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1002?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJ4%2FcHxhLMCcp1w
no negative reviews for this game
1200 4494
https://store.steampowered.com/appreviews/1200?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1200?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4wIT7k%2BkCc4LruwE%3D
https://store.steampowered.com/appreviews/1200?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwqv%2BM%2FNUCdOz6XA%3D%3D
https://store.ste

https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw6JvgwM0CdOj0PA%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwpL2shs0Cf%2BvPOw%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4gfuH6swCcLD7Og%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwp%2F%2Bsx8wCf9COOg%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ43Lvir8wCd4jHOQ%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw7qyWoswCebijOQ%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw4eT1ncwCcIKVOQ%3D%3D
https://store.steampowered.c

https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw%2F5jp6sQCfv2yJA%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ457XTw8QCcvLKIw%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwwpKopsQCeseFIw%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4wLHOj8QCcaPDIg%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwyLuY78MCfdb5IQ%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwvKOr0MMCcu2rIQ%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4tfe1uMMCcpLvIA%3D%3D
https://store.steampowered.com/a

https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwlZeUwrQCcZeABw%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw6J3wwLQCd9W2BQ%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwl8XCwLQCfca4Ew%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4nequwLQCerWsDA%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4s66mtrQCddfiBg%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4ms79m7QCfcjgBA%3D%3D
https://store.steampowered.com/appreviews/1250?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwmqrbg7QCdKHMCQ%3D%3D
https://store.steampowered.com/app

no negative reviews for this game
1313 4501
https://store.steampowered.com/appreviews/1313?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1313?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4ttK7tawCcsVj
no positive reviews for this game
https://store.steampowered.com/appreviews/1313?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1313?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJ41%2F3MtbYCctpS
no negative reviews for this game
1500 4502
https://store.steampowered.com/appreviews/1500?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=*
https://store.steampowered.com/appreviews/1500?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwj%2FitztgCeq37aA%3D%3D
https://store.steampowered.com/appreviews/1500?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwwPDT97sCcOKhCg

...

https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwm%2Bmn%2FNMCcM6NVQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwpuev4tMCfJO3VA%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4o87PytMCd4TeUw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwq%2BGBsdMCdsX9Ug%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwkseSmNMCcJieUg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ45vjdgtMCcuXDUQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw1f6J8tICfa38UA%3D%3D
https://store.steampo

https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwnfjYz8sCf6y6Nw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw7L%2FAussCe4mCNw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4iL3lpssCdLbJNg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwy92ClcsCdduPNg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4u4auhcsCfIXhNQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwmZmX98oCfI%2ByNQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ45sPh58oCfpyINQ%3D%3D
https://store.steampowe

https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw%2BsDfmcYCd5CDJw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ4qv2H9MUCcqK4Jg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJw7L2ay8UCe%2BntJQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwnoOrpcUCevelJQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJ41K7b%2FsQCeJ3fJA%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwkoDE2MQCfob3Iw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=positive&num_per_page=100&cursor=AoJwwLHUucQCdeCyIw%3D%3D
https://store.steampo

https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJw96Sm9%2BECeerXmgE%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJ4kvaZ%2FtwCfKySfg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJ4qfjPxNgCe5eAZg%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJw7oCx%2B9ICeYakUQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJw%2B8XT4c4CftTVQQ%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJwotOitcoCfK25Mw%3D%3D
https://store.steampowered.com/appreviews/48700?json=1&filter=recent&review_type=negative&num_per_page=100&cursor=AoJwyeuTtMYCc8i8Jw%3D%3D
https://store.steampo

KeyboardInterrupt: 

In [384]:
#run this cell if the reviews collection script is interrupted
anotherlist = []
for game in np.array(df['game_id']):
    if game not in done_games:
        anotherlist.append(game)